You will 1st need to upload data.zip.Then run the below cell to preprocess images

In [0]:
!unzip "data.zip"

In [0]:
import os
import cv2
import pandas as pd
import time

In [0]:
root_dir = os.getcwd()
file_list = ['train.csv', 'val.csv']
image_source_dir = os.path.join(root_dir, 'data/images/')
data_root = os.path.join(root_dir, 'data')
os.chdir(data_root)
for file in file_list:
    flag=0
    print(file)
    image_target_dir = os.path.join(data_root, file.split(".")[0])
    # read list of image files to process from file
    image_list = pd.read_csv(os.path.join(data_root, file), header=0)['image_id']
    print("Start preprocessing images")
    for image in image_list:
        if(image_target_dir.split("/")[len(image_target_dir.split("/"))-1]=="train" and flag==0):
            os.mkdir("train")
            os.chdir(data_root+"/train")
            flag=1
        elif(image_target_dir.split("/")[len(image_target_dir.split("/"))-1]=="val") and flag==0:
            os.chdir(data_root)
            os.mkdir("val")
            os.chdir(data_root+"/val")
            flag=2
        # open image file
        img = cv2.imread(os.path.join(image_source_dir, image))
        print(image_source_dir,image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # perform transformations on image
        b = cv2.distanceTransform(img, distanceType=cv2.DIST_L2, maskSize=5)
        g = cv2.distanceTransform(img, distanceType=cv2.DIST_L1, maskSize=5)
        r = cv2.distanceTransform(img, distanceType=cv2.DIST_C, maskSize=5)
        # merge the transformed channels back to an image
        transformed_image = cv2.merge((b, g, r))
        target_file = os.path.join(image_target_dir, image)
        print("Writing target file {}".format(target_file))
        cv2.imwrite(image, transformed_image)

In [0]:
os.chdir("/content")

In [0]:
!cp data/config.yml config.yml

In [0]:
!pip install luminoth

    100% |████████████████████████████████| 225kB 15.5MB/s 
    100% |████████████████████████████████| 71kB 22.5MB/s 
    100% |████████████████████████████████| 2.3MB 1.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/3f/f0/7c/1ec164f1dc11a6e0ef58463f5e25e83d9c6f661e2e8e70bcef
Successfully built easydict
distributed 1.26.0 has requirement tornado>=5, but you'll have tornado 4.5.3 which is incompatible.
  Found existing installation: easydict 1.9
    Uninstalling easydict-1.9:
      Successfully uninstalled easydict-1.9
  Found existing installation: Click 7.0
    Uninstalling Click-7.0:
      Successfully uninstalled Click-7.0


In [0]:
!lumi dataset transform --type csv --data-dir data/ --output-dir tfdata/ --split train --split val --only-classes=table


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Saving split "train" in output_dir = tfdata/
  [####################################]  100%             
INFO:tensorflow:Saved 338 records to "tfdata/train.tfrecords"
INFO:tensorflow:Composition per class (train):
INFO:tensorflow:	table: 418
INFO:tensorflow:Saving split "val" in output_dir = tfdata/
  [####################################]  100%
INFO:tensorflow:Saved 65 records to "tfdata/val.tfrecords"
INFO:tensorflow:Composition per class (val):
INFO:tensorflow:	table: 100


In [0]:
os.chdir("/content")

In [0]:
!lumi train -c config.yml


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Colocations handled automatically by placer.
Instructions f

In [0]:
!lumi checkpoint create config.yml


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Creating checkpoint for given configuration...
Checkpoint ed1d749be67e created successfully.


In [0]:
predix=!lumi predict --checkpoint ed1d749be67e data/val/9541_023.png

In [0]:
predix[-1]

'{"file": "data/val/9541_023.png", "objects": [{"bbox": [127, 608, 2315, 2156], "label": "table", "prob": 0.9795}, {"bbox": [1355, 590, 2346, 2175], "label": "table", "prob": 0.8494}, {"bbox": [156, 1440, 2247, 2164], "label": "table", "prob": 0.787}, {"bbox": [206, 638, 2226, 1403], "label": "table", "prob": 0.6043}, {"bbox": [100, 1855, 2304, 2149], "label": "table", "prob": 0.5714}, {"bbox": [207, 732, 1263, 2106], "label": "table", "prob": 0.5401}]}'

In [0]:
val=pd.read_csv("data/val.csv")

In [0]:
!lumi server web --checkpoint ed1d749be67e


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

 * Serving Flask app "luminoth.tools.server.web" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
2019-03-02 02:42:12.383605: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-02 02:42:12.383853: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x17c8260 executing computations on platform Host. Devices:
2019-03-02 02:42:12.383890: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2019-03-02 02:42:12.488224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but th

In [0]:
for i in range(1):#len(val['image_id'])):
  predix=!lumi predict --checkpoint ed1d749be67e data/val/val['image_id'][5]